In [1]:
cd ../..

/home/neon/Documents/cwi_assignament


# Assuigment: Generative AI for Insight Retrieval from Structured Data
#### Cristobal Donoso Oliva 
##### Nov 27th

In [2]:
import pandas as pd
import numpy as np
import chromadb
import pickle
import ast
import os

from src.pipelines.naive import naive_pipeline
from src.encode import encode_corpus_query, encode_from_documents
from src.utils import format_table, get_accuracy, fix_duplicated_columns


%load_ext autoreload
%autoreload 2


/home/neon/miniconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We first load the `corpus.parquet` and `queries.parquet` files, which contain the tables and desired queries to be answered

In [3]:
corpus = pd.read_parquet('./data/corpus.parquet')
queries = pd.read_parquet('./data/queries.parquet')
corpus.shape

(1001, 4)

Since pandas dataframes that have been saved lose their data types, it is necessary to format each row to be a proper table.

In [4]:
corpus['table'] = corpus['table'].apply(lambda x: format_table(x)) 
corpus.iloc[2]['table']

,Year,Award,Nominee,Category,Result
0,2013,DJ Magazine Awards,Dyro,Top 100 DJs,30
1,2014,DJ Magazine Awards,Dyro,Top 100 DJs,27
2,2015,DJ Magazine Awards,Dyro,Top 100 DJs,27
3,2016,DJ Magazine Awards,Dyro,Top 100 DJs,93


Here is an example of a Query/Answer tuple from our data:

In [5]:
idx = queries.sample()['database_id'].values[0]
print('Q:{}\nA:{}'.format(queries[queries['database_id'] == idx]['query'].values[0],
queries[queries['database_id'] == idx]['answer'].values[0]))

Q:Who representing what nation finished fifth and sixth in the swimming at the 2012 Summer Olympics final -- Women's 800 metre freestyle and at what are their times?
A:In the final Women's 800 metre freestyle, Denmark's Lotte Friis finished fifth in 8:23.86 and Hungarian Boglárka Kapás sixth in 8:23.89.


## RAG PIPELINES


In the following sections we are going to use our custom pipelines, but first of all we need to check for errors in table:

In [6]:
corpus['table'] = corpus['table'].apply(lambda x: fix_duplicated_columns(x))

In [7]:
MAXLENTOKEN = [len(str(x.to_html())) for x in corpus['table']]
print('Maximum table length: ', np.max(MAXLENTOKEN))

Maximum table length:  11157


In the following cells, we split tables into records or table rows. This is useful for controlling the maximum number of tokens while preserving contextual and semantic information (most LLMs truncate the input when they exceed the number of tokens).

Even though this approach seems to be better than using all the tables, we didn't obtain better performance and it also increases the computation time.

In [8]:
documents = []
for i, row in corpus.iterrows():
    values = row['table'].to_dict(orient='records')
    values = [str({**{'table_id': row['database_id']}, **x, **row['context']}) for x in values]
    documents+=values

In [9]:
doclen = [len(d) for d in documents]
i = np.argmax(doclen)
nless = np.sum(np.less(doclen, 256))

print('Max token lenght: {}\nNumber of documents with lenght less than 384: {}/{} ({:.2f}%)'.format(max(doclen), 
                                                                                          nless, 
                                                                                          len(doclen),
                                                                                          nless/len(doclen)*100
                                                                                          ))

Max token lenght: 1823
Number of documents with lenght less than 384: 9223/13678 (67.43%)


### 1.1 Getting embeddings

For getting embeddings we transform tables to HTML strings. We chose HTML since LLM are often trained on web content, so it is likely that they are more familiar with this format.

Context information from `corpus` database is also important. We encode `corpus['table']` and `corpus['context']` separately, then we average both representation to create the final embedding

### Naive

In [20]:
naive_embeddings, naive_queries_embeddings = encode_corpus_query(corpus, queries) 

In [11]:
ids = corpus['database_id'].astype(str).to_list()

naive_top_five = naive_pipeline(naive_embeddings, naive_queries_embeddings, dbids=np.array(ids, dtype='int'))
'Naive solution Accuracy: {}'.format(np.mean(get_accuracy(queries, naive_top_five)))

'Naive solution Accuracy: 0.74'

### Chunks-based solution

In [17]:
stored_file = './data/embeddings.pickle'
if os.path.exists(stored_file):
    with open(stored_file, 'rb') as file:
        chunks = pickle.load(file)
else:
    embeddings, queries_embeddings = encode_from_documents(documents, queries) 
    with open('./data/embeddings.pickle', 'wb') as file:
        chunks = {'doc_emb': embeddings, 
                   'qry_emb':queries_embeddings, 
                   'documents':documents, 
                   'queries':queries}
        pickle.dump(chunks, file)

In [19]:
chunks_top_five = naive_pipeline(chunks['doc_emb'], chunks['qry_emb'], dbids=np.array(ids, dtype='int'))
'Chunks-based solution Accuracy: {}'.format(np.mean(get_accuracy(queries, chunks_top_five)))

'Chunks-based solution Accuracy: 0.44'

## Indexation

### Chroma

We use ChromaDB for indexing. Here we use the embeddings obtained by the naive version.

In [24]:
chroma_client = chromadb.Client()
try:
    collection = chroma_client.create_collection(name="vec_db", metadata={"hnsw:space": "cosine"} )
except:
    chroma_client.delete_collection("vec_db")
    collection = chroma_client.create_collection(name="vec_db", metadata={"hnsw:space": "cosine"} )

collection.add(
    embeddings=naive_embeddings,
    ids=corpus['database_id'].astype(str).to_list())

In [25]:
results = collection.query(query_embeddings=queries_embeddings, n_results=5)

In [26]:
accval = np.mean(get_accuracy(queries, np.array(results['ids'], dtype='int')))
'Chroma solution Accuracy: {:.2f}'.format(accval)

'Chroma solution Accuracy: 0.62'

Even though we use exactly the same embeddings the accuracy decreased. It can be the effect of the searching algorithm used by Chroma or the distance metric it is using